# Veo3 Simple Prompt-to-Video (Standalone)

This notebook lets you enter your own prompt, generate a video using Google's Veo3 Fast via the Gemini API, and play it inline. It is fully standalone and uses your `GOOGLE_API_KEY` directly; no project-specific configuration is required.

## Requirements

- `GOOGLE_API_KEY` set in your environment (AI Studio key)
- `google-genai` and `ipywidgets` installed (see the next cell)

Tip: Veo is a paid feature. Charges may apply according to your account's pricing plan.


In [6]:
# Install dependencies (uncomment if needed)
# %pip install -U google-genai ipywidgets python-dotenv



In [7]:
# Standard library
import os
import time
import mimetypes

# Third-party
from dotenv import load_dotenv
from IPython.display import display, HTML, Video, clear_output
import ipywidgets as widgets
from google import genai
from google.genai import types

# Load environment (supports .env too)
load_dotenv()

print("✅ Libraries imported")


✅ Libraries imported


In [8]:
# Initialize client directly from GOOGLE_API_KEY
print("🔧 Initializing Gemini client...")
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise RuntimeError("GOOGLE_API_KEY is not set. Set it in your environment or .env and restart.")

client = genai.Client(api_key=api_key)
VEO_MODEL_ID = "veo-3.0-fast-generate-preview"
DEFAULT_ASPECT = "16:9"  # Veo 3 Fast supports 16:9 per docs

print("✅ Client ready")
print(f"🎬 Veo Model: {VEO_MODEL_ID}")
print(f"📐 Aspect Ratio: {DEFAULT_ASPECT}")


🔧 Initializing Gemini client...
✅ Client ready
🎬 Veo Model: veo-3.0-fast-generate-preview
📐 Aspect Ratio: 16:9


In [ ]:
class SimpleVeoVideoGenerator:
    def __init__(self, client):
        self.client = client

    def generate(
        self,
        *,
        prompt: str,
        model: str,
        negative_prompt: str | None = None,
        image: types.Image | None = None,
        progress=None,
    ):
        if not prompt or not prompt.strip():
            raise ValueError("Prompt must not be empty")

        if progress:
            progress("🚀 Starting video generation...")

        # NOTE: `person_generation` is intentionally omitted from the
        # configuration because certain values (e.g. "dont_allow") currently
        # cause the API to return a 400 INVALID_ARGUMENT:
        #   {'error': {'code': 400, 'message': 'dont_allow for personGeneration is currently not supported.', 'status': 'INVALID_ARGUMENT'}}
        # Re-introduce person_generation handling here once the Veo API
        # supports it.
        cfg_kwargs = dict(
            aspect_ratio=DEFAULT_ASPECT,
            number_of_videos=1,
        )
        if negative_prompt:
            cfg_kwargs["negative_prompt"] = negative_prompt

        operation = self.client.models.generate_videos(
            model=model,
            prompt=prompt,
            image=image,
            config=types.GenerateVideosConfig(**cfg_kwargs),
        )

        # Poll the long-running operation with a bounded deadline to avoid
        # indefinite hanging. Allow configurable timeout and poll interval.
        start = time.time()
        timeout = 300  # seconds, adjust as needed (5 minutes default)
        poll_interval = 5

        # The operations.get method accepts either the operation object or its
        # name/identifier; normalize to the name if present so repeated calls
        # refresh status reliably.
        op_ref = getattr(operation, "name", operation)

        while True:
            # Refresh operation status
            operation = self.client.operations.get(op_ref)
            done = getattr(operation, "done", False)
            if done:
                break

            elapsed = time.time() - start
            if elapsed >= timeout:
                raise TimeoutError(f"Generation did not complete within {timeout:.0f}s (operation={op_ref})")

            if progress:
                progress(f"⏳ Generating... elapsed {elapsed:.0f}s")

            time.sleep(poll_interval)

        # If the operation ended with an error, raise a clear exception.
        if getattr(operation, "error", None):
            err_msg = getattr(operation, "error", None)
            raise RuntimeError(f"Generation failed: {getattr(err_msg, 'message', 'Unknown error')}")

        video_data = operation.result.generated_videos[0]

        # Download/cache then save to get bytes
        self.client.files.download(file=video_data.video)
        tmp_name = f"generated_video_{int(time.time())}.mp4"
        video_data.video.save(tmp_name)
        with open(tmp_name, "rb") as f:
            video_bytes = f.read()
        os.remove(tmp_name)

        return {
            "video_bytes": video_bytes,
            "operation_id": operation.name,
            "elapsed": time.time() - start,
            "aspect_ratio": DEFAULT_ASPECT,
            "prompt": prompt,
            "model": model,
            "negative_prompt": negative_prompt or "",
        }

    def save(self, video_bytes: bytes, filename: str | None = None) -> str:
        if not video_bytes:
            raise ValueError("No bytes to save")
        name = filename or f"generated_video_{int(time.time())}.mp4"
        with open(name, "wb") as f:
            f.write(video_bytes)
        return name

    def display(self, video_bytes: bytes, width: int = 720) -> str:
        path = self.save(video_bytes)
        display(Video(path, embed=True, width=width))
        return path

video_gen = SimpleVeoVideoGenerator(client)
print("✅ Video generator ready")


✅ Video generator ready


In [ ]:
# UI: Prompt input + flexible controls
SAMPLE_PROMPTS = [
    "Cinematic wide shot of a misty pine forest at sunrise; slow push-in; golden backlight through fog; shallow depth of field; crisp details on dew; natural ambience with distant birdsong.",
    "Handheld medium shot inside a cozy coffee shop; warm tungsten lighting; rack focus from latte art to barista; gentle background chatter and soft jazz.",
    "Aerial establishing shot over a coastal cliff at golden hour; slow orbit; dramatic waves crashing; wide dynamic range and clean highlights; natural ocean ambience."
]

prompt_input = widgets.Textarea(
    value=SAMPLE_PROMPTS[0],
    placeholder="Describe the scene you want... Include shot type, camera move, lighting, and style.",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="80px")
)

sample_dropdown = widgets.Dropdown(
    options=[("Choose a sample prompt…", "")] + [(p[:80] + ("…" if len(p) > 80 else ""), p) for p in SAMPLE_PROMPTS],
    value="",
    description="Samples:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="100%")
)

def on_sample_change(change):
    if change["name"] == "value" and change["new"]:
        prompt_input.value = change["new"]
        sample_dropdown.value = ""

sample_dropdown.observe(on_sample_change, names="value")

model_dropdown = widgets.Dropdown(
    options=[
        ("Veo 3 Fast (recommended)", "veo-3.0-fast-generate-preview"),
        ("Veo 3 (higher quality)", "veo-3.0-generate-preview"),
        ("Veo 2 (legacy, no audio)", "veo-2.0-generate-001"),
    ],
    value=VEO_MODEL_ID,
    description="Model:",
    style={'description_width': 'initial'}
)

negative_input = widgets.Text(
    value="",
    placeholder="What to avoid (e.g., text overlays, low quality)",
    description="Negative:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="100%")
)

image_upload = widgets.FileUpload(
    accept="image/*",
    multiple=False,
    description="Upload image (optional)"
)

ack_checkbox = widgets.Checkbox(
    value=False,
    description="I acknowledge Veo is a paid feature",
    indent=False
)

generate_btn = widgets.Button(
    description="🎬 Generate Video",
    button_style="success",
    layout=widgets.Layout(width="200px")
)

progress_out = widgets.Output()
video_out = widgets.Output()


def _build_image_from_upload(uploader: widgets.FileUpload) -> types.Image | None:
    if not uploader.value:
        return None
    # Support ipywidgets v7 (tuple of dicts) and v8 (dict mapping)
    file_obj = None
    if isinstance(uploader.value, dict):
        # v8 format: {filename: {"content": bytes, "metadata": {"name":..., "type":...}}}
        key = next(iter(uploader.value))
        entry = uploader.value[key]
        content = entry.get("content")
        mime = entry.get("metadata", {}).get("type") or entry.get("type")
        name = entry.get("metadata", {}).get("name", key)
        if not mime and name:
            mime = mimetypes.guess_type(name)[0]
        return types.Image(image_bytes=content, mime_type=mime or "image/png")
    else:
        # v7 format: tuple of dicts with keys: 'name','type','size','content'
        file_obj = next(iter(uploader.value))
        content = file_obj.get("content")
        mime = file_obj.get("type")
        name = file_obj.get("name")
        if not mime and name:
            mime = mimetypes.guess_type(name)[0]
        return types.Image(image_bytes=content, mime_type=mime or "image/png")


def on_generate_clicked(_):
    with progress_out:
        clear_output(wait=True)
        print("Validating input...")
    with video_out:
        clear_output(wait=True)

    if not ack_checkbox.value:
        with progress_out:
            print("⚠️ Please acknowledge that Veo is a paid feature before proceeding.")
        return

    text = prompt_input.value.strip()
    if not text:
        with progress_out:
            print("❌ Please enter a prompt")
        return

    selected_model = model_dropdown.value
    negative = negative_input.value.strip() or None
    img = _build_image_from_upload(image_upload)

    def report(msg: str):
        with progress_out:
            print(msg)

    try:
        result = video_gen.generate(
            prompt=text,
            model=selected_model,
            negative_prompt=negative,
            image=img,
            progress=report,
        )
        with video_out:
            print("\n🎉 Generation complete!")
            print(f"⏱️ Time: {result['elapsed']:.1f}s | Aspect: {result['aspect_ratio']}")
            if result["negative_prompt"]:
                print(f"🚫 Negative: {result['negative_prompt']}")
            path = video_gen.display(result["video_bytes"], width=720)
            print(f"💾 Saved: {path}")
    except Exception as e:
        with video_out:
            print(f"❌ Error: {e}")


generate_btn.on_click(on_generate_clicked)

# Layout
controls_row = widgets.HBox([model_dropdown])
advanced_box = widgets.VBox([negative_input, image_upload])

display(HTML("<h3>Enter Prompt</h3>"))
display(sample_dropdown)
display(prompt_input)
display(controls_row)
display(advanced_box)
display(ack_checkbox)
display(generate_btn)
display(progress_out)
display(HTML("<h3>Output</h3>"))
display(video_out)


Dropdown(description='Samples:', layout=Layout(width='100%'), options=(('Choose a sample prompt…', ''), ('Cine…

Textarea(value='Cinematic wide shot of a misty pine forest at sunrise; slow push-in; golden backlight through …

Checkbox(value=False, description='I acknowledge Veo is a paid feature', indent=False)

Button(button_style='success', description='🎬 Generate Video', layout=Layout(width='200px'), style=ButtonStyle…

Output()

Output()

## Notes and Best Practices

- Standalone: reads `GOOGLE_API_KEY` and initializes the Gemini client directly.
- Flexibility: choose model (Veo 3 Fast / Veo 3 / Veo 2), optionally add a negative prompt, and optionally upload an image for image-to-video.
- People policy: `person_generation` is intentionally omitted due to known API constraints that return INVALID_ARGUMENT for certain values; re-enable when supported by the Veo API.
- Aspect ratio: fixed to `16:9` to align with Veo 3 Fast guidance.
- The helper downloads and saves the video before displaying (per official examples).
- Set `GOOGLE_API_KEY` via your environment or a `.env` file. Examples:
  - macOS/Linux: `export GOOGLE_API_KEY="your-key"`
  - Windows (PowerShell): `$Env:GOOGLE_API_KEY="your-key"`
- Costs: Veo usage is billed. Review pricing and your quotas.
- Troubleshooting:
  - API key validity
  - Model availability in your region
  - Prompt content safety (no prohibited content)

You're ready to experiment: enter a prompt, pick options, and click Generate.
